In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
#Kütüphanelerin eklenmesi
import numpy as np #Bu kütüphane lineer cebir için kullandığımız kütüphane fonksiyonlarını içeriyor
import pandas as pd # verilerimizi işlemek için pandas kütüphanasini kullanıyoruz(örn pd.read_scv)
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, train_test_split, cross_val_predict, cross_val_score
import matplotlib.pyplot as plt
%matplotlib inline
!pip install utils
!pip install zemberek-python
from utils import *
import json
import random
#from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish
import warnings
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
import re
import string
warnings.filterwarnings('ignore') 
from sklearn.metrics import classification_report, accuracy_score, make_scorer


from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import *

# DEEP LEARNING IMPORTS
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv1D, Activation, Dropout, Flatten, MaxPooling1D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix


from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from glove import Corpus, Glove

#https://medium.com/analytics-vidhya/word-vectorization-using-glove-76919685ee0b
#https://github.com/maciejkula/glove-python



In [ ]:
#fasttext modelini git üzerinden çekiyoruz

##NOT: JUPYTER C DİSKİ ÜZERİNDE ÇALIŞTIĞI VE DİSKİN DOLU OLMASINDAN DOLAYI VE FASTTEXT
#MODELİNİN KAPLADIĞI ALAN YÜZÜNDEN KODU GOOGLE COLAB ÜZERİNDE ÇALIŞTIRDIM O YÜZDEN DOSYA İÇİNDE 
#FASTTEXT.GİT KLASÖRÜ OLMAYACAKTIR. KODLAR COLAB ÜZERİNDEN İSTENİLİRSE DERLENEBİLİR

#!git clone https://github.com/facebookresearch/fastText.git

#!pip install fasttext

In [ ]:
import fasttext

In [ ]:
import fasttext.util
#Türkçe için önceden eğitilmiş kelime vektörlerini alıyoruz
#fasttext.util.download_model('tr', if_exists='ignore')

In [ ]:
#modeli yüklüyoruz
ft = fasttext.load_model('D:/Desktop/İndirilenler yedek/cc.tr.300.bin')

In [ ]:
column = ['tweets','duygu','preprocessing']
df = pd.read_excel("../dataset/total.xlsx")
df.columns=column
#veri setinin gösterilmesi
df=df.drop_duplicates()
df['tweets']=df['tweets'].str.lower()


In [ ]:
df=df.sample(frac=1).reset_index(drop=True)

In [ ]:
def classification_report_with_accuracy_score(y_true, y_pred):
    print(classification_report(y_true, y_pred)) # print classification report
    
    return accuracy_score(y_true, y_pred) # return accuracy score

In [ ]:
def classification_report_with_precision_score(y_true, y_pred):
    
    return precision_score(y_true, y_pred, average='weighted')

In [ ]:
def classification_report_with_accuracy_score(y_true, y_pred):
    print(classification_report(y_true, y_pred)) # print classification report
    return accuracy_score(y_true, y_pred) # return accuracy score

In [ ]:
def classification_report_with_recall_score(y_true, y_pred):
    
    return recall_score(y_true, y_pred,average='weighted')

In [ ]:
def classification_report_with_f1_score(y_true, y_pred):
    
    return f1_score(y_true, y_pred,average='weighted')

In [ ]:
WPT = nltk.WordPunctTokenizer()
#verimize preprocessing aşamalarını uyguluyoruz
def feature_extraction_fasttext(text):
    sentence=""
    sentence=str(text) #alınan veriyi string çeviriyoruz
    #stop_word_list = nltk.corpus.stopwords.words('turkish') #türkçe için çok tekrar eden kelimeleri (stopwords) cümlelerden çıkarıyoruz
    tokens=WPT.tokenize(sentence) #cümleyi kelimelere bölüyoruz
    new_tokens=[token for token in tokens ]#if token not in stop_word_list] #stopword listesinde olan kelimeleri çıkarıyoruz
    sent_list=[]

    for word in new_tokens:
        wv=ft.get_word_vector(word) #fasttext modeline kelimeyi yolluyorum
        sent_list.append(wv)
 
    sent_embed= np.mean(sent_list,axis=0) # fasttext gösterimlerinin ortalamasını alarak cümle vektörünü çıktı olarak veriyor
    return sent_embed


In [ ]:
y_total= []
y_total=df.duygu.tolist()

X_total = []
for element in tqdm(df.tweets):
    X_total.append(feature_extraction_fasttext(element))

In [ ]:
# Nested CV with parameter optimization
clf = svm.SVC(kernel='linear')

accuracy = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_accuracy_score))
print("Accurancy for fasttext/SVM: ",accuracy.mean())


precision = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_precision_score))
print("Precision for fasttext/SVM: ",precision.mean())


recall = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_recall_score))
print("Recall for fasttext/SVM: ",recall.mean())


f1 = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_f1_score))
print("F1-Score for fasttext/SVM: ",f1.mean())

In [ ]:
# Nested CV with parameter optimization
clf = LogisticRegression()

accuracy = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_accuracy_score))
print("Accurancy for fasttext/LogisticRegression: ",accuracy.mean())


precision = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_precision_score))
print("Precision for fasttext/LogisticRegression: ",precision.mean())


recall = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_recall_score))
print("Recall for fasttext/LogisticRegression: ",recall.mean())


f1 = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_f1_score))
print("F1-Score for fasttext/LogisticRegression: ",f1.mean())


In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(10, 3), random_state=1)

accuracy = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_accuracy_score))
print("Accurancy for fasttext/MLPC: ",accuracy.mean())


precision = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_precision_score))
print("Precision for fasttext/MLPC: ",precision.mean())


recall = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_recall_score))
print("Recall for fasttext/MLPC: ",recall.mean())


f1 = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_f1_score))
print("F1-Score for fasttext/MLPC: ",f1.mean())
